In [1]:
# Script to calculate convective environmental parameters from mean profiles that are read in

# Load Libraries
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from metpy.units import units
import numpy as np
import matplotlib as mpl
from collections import OrderedDict
import glob
import os

/home/pmarin/miniconda3/envs/basic/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Function to calculate the mean specific humidity between two pressure levels
def calc_mean_layer_p(alt_arr,pres_lvls,data):

    # Get pressure level indicies
    alt0 = np.where(np.abs(np.array(pres_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(pres_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    # Get specific data and do calculation
    var1 = data[int(alt0[0]):int(alt1[0])]
    var = np.nanmean(var1)
    del(var1)
    
    return var

# Function to calculate the wind shear between two pressure levels
def calc_layer_shear_p(alt_arr,pres_lvls,udata,vdata):
    #### LOW LEVEL SHEAR        
#    alt_arr = [800, 1000]
    alt0 = np.where(np.abs(np.array(pres_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(pres_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    var1 = udata[int(alt0[0])]
    var2 = vdata[int(alt0[0])]
    var3 = udata[int(alt1[0])]
    var4 = vdata[int(alt1[0])]
    var = np.sqrt(np.power((var3-var1),2.0) + np.power((var4-var2),2.0))
    return var

# Function to calculate the wind shear between two altitude levels
def calc_layer_shear_z(alt_arr,alt_lvls,udata,vdata):
    #### LOW LEVEL SHEAR        
#    alt_arr = [800, 1000]
    alt0 = np.where(np.abs(np.array(alt_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(alt_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    var1 = udata[int(alt0[0])]
    var2 = vdata[int(alt0[0])]
    var3 = udata[int(alt1[0])]
    var4 = vdata[int(alt1[0])]
    var = np.sqrt(np.power((var3-var1),2.0) + np.power((var4-var2),2.0))   
    return var

# Function to calculate the lapse rate between two pressure levels
def calc_lapse_rate_p(alt_arr,pres_lvls,temp,zmn):
    alt0 = np.where(np.abs(np.array(pres_lvls) - alt_arr[0]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(pres_lvls) - alt_arr[1]) == np.min(np.abs(np.array(pres_lvls) - alt_arr[1])))

    var1 = temp[int(alt0[0])]
    var2 = temp[int(alt1[0])]
    var3 = zmn[int(alt0[0])]
    var4 = zmn[int(alt1[0])]
    var = -(var2-var1)/(var4-var3)   
    return var

# Function to calculate total column water vapor
def calc_tcwv(pres,temp,rv,zmn,topt):
    rd = 287; #J/kg/K
    dens = pres/(rd*temp*(1+0.61*rv)) # calculate dense
    del(pres,temp)
    
    # Difference in heights (dz)    
    diff_zt = np.diff(zmn)*(1-topt/np.nanmax(zmn))
    
    # integrated total column water vapor in kg / m^2
    tcwv = np.nansum(rv[1:]*dens[1:]*diff_zt)
    del(rv,diff_zt,dens)

    return tcwv    

In [3]:
#Function to calculate CAPE
from __future__ import print_function
import numpy as np

def getcape(p_in, t_in, td_in, source_parc = 'mu', ml_depth = 300.0, adiabat = 'pl'):
    '''
    Ported from George Bryan's code: http://www2.mmm.ucar.edu/people/bryan/Code/getcape.F
    p_in: 1D pressure array (mb)
    t_in: 1D temperature array (C)
    td_in: 1D dewpoint array (C)

    Optional: 
    source_parc: source parcel (mu: most unstable, 
                                sb: surface, ml: mixed layer)
    ml_depth: depth of mixed layer (m)
    adiabat: Formulation of moist adiabat (pl: pseudoadiabatic, liquid only,
        rl: reversible, liquid only, pi: pseudoadiabatic, with ice, 
        ri: reversible, with ice)

    Returns:
    cape: Convective Available Potential Energy (J/kg)
    cin: Convective Inhibition (J/kg)
    '''
    cape=0
    cin = 0

    pinc = 100.0 #pressure increment (Pa)


    #defining constants:
    g     = 9.81
    p00   = 100000.0
    cp    = 1005.7
    rd    = 287.04
    rv    = 461.5
    xlv   = 2501000.0
    xls   = 2836017.0
    t0    = 273.15
    cpv   = 1875.0
    cpl   = 4190.0
    cpi   = 2118.636
    lv1   = xlv+(cpl-cpv)*t0
    lv2   = cpl-cpv
    ls1   = xls+(cpi-cpv)*t0
    ls2   = cpi-cpv

    rp00  = 1.0/p00
    eps   = rd/rv
    reps  = rv/rd
    rddcp = rd/cp
    cpdrd = cp/rd
    cpdg  = cp/g

    converge = 0.002
    debug_level=0


    #convert p, t, td to mks units, get pi, q, th, thv
    p = 100*np.array(p_in)
    t = 273.15+np.array(t_in)
    td = 273.15+np.array(td_in)
    pi = np.power(p*rp00, rddcp)
    q = getqvs(p,td)
    th = t/pi
    thv = th*(1.0+reps*q)/(1.0+q)

    #get hydrostatic heights
    z = np.zeros(len(p))

    for i in range(1,len(z)):
        dz = -cpdg*0.5*(thv[i]+thv[i-1])*(pi[i]-pi[i-1])
        z[i] = z[i-1] + dz


    if source_parc == 'sb':
        #surface parcel
        kmax = 0
    elif source_parc == 'mu':
        #use most unstable parcel
        if p[0] < 50000.0:
            #first pressure is above 500 mb, instead use the level reported.
            kmax = 0
            maxthe = getthe(p[0], t[0], td[0], q[0])
        else:
            #find max theta e below 500 mb.

            p500 = p>50000.0
            maxthe = 0.0
            the500 = list()
            for k in range(len(p)):
                if p[k] >= 50000.0:
                    the = getthe(p[k],t[k],td[k],q[k])
                    if the > maxthe:
                        maxthe = the
                        kmax = k

    elif source_parc == 'ml':
        #mixed layer.

        if (z[1] - z[0]) > ml_depth:
            #second level is above the ml depth, use the lowest level
            avgth = th[0]
            avgqv = q[0]
            kmax = 0

        elif z[-1] < ml_depth:
            #topmost level is within the mixed layer: 
            #use the upper-most level
            avgth = th[-1]
            avgqv = q[-1]
            kmax = len(th)-1

        else:
            #calculate the ml properties
            avgth = 0
            avgqv = 0
            k = 1

            while z[k] <= ml_depth and k <= len(z)-1:
                avgth = avgth + 0.5*(z[k] - z[k-1])*(th[k]+th[k-1])
                avgqv = avgqv + 0.5*(z[k] - z[k-1]) * (q[k] + q[k-1])
                k = k+1

            th2 = th[k-1] + (th[k] - th[k-1])*(ml_depth - z[k-1])/(z[k] - z[k-1])
            qv2 = q[k-1] + (q[k] - q[k-1]) * (ml_depth - z[k-1])/(z[k] - z[k-1])

            avgth = avgth + 0.5*(ml_depth-z[k-1]) * (th2+th[k-1])
            avgqv = avgqv + 0.5*(ml_depth - z[k-1])* (qv2+q[k-1])

            avgth = avgth/ml_depth
            avgqv = avgqv/ml_depth

            kmax = 0
    else:
        raise ValueError("Source Parcel can be ml, sb, or mu only.")

    narea = 0.0

    if source_parc == 'sb' or source_parc == 'mu':
        k = kmax
        th2 = th[kmax]
        pi2 = pi[kmax]
        p2 = p[kmax]
        t2 = t[kmax]
        thv2 = thv[kmax]
        qv2 = q[kmax]
        b2 = 0.0


    elif source_parc == 'ml':
        k = kmax
        th2 = avgth
        qv2 = avgqv
        thv2 = th2 * (1.0+reps*qv2)/(1.0+qv2)
        pi2 = pi[kmax]
        p2 = p[kmax]
        t2 = th2 * pi2
        b2 = g * (thv2 - thv[kmax])/thv[kmax]

    ql2 = 0.0
    qi2 = 0.0
    qt = qv2
    cape = 0.0
    cin = 0.0 
    lfc = 0.0

    doit = True
    cloud = False
    if 'i' in adiabat:
        ice=True
    else:
        ice = False

    the = getthe(p2,t2,t2,qv2)
    #begin ascent of parcel

    if debug_level >=100:
        print('Start Loop:')
        print('p2, th2, qv2', p2, th2, qv2)

    while doit and k <len(z)-1:
        k = k+1
        b1 = b2
        try:
            dp = p[k-1] - p[k]
        except IndexError:
            print(k, p[k-1], len(z))
            return
        if dp < pinc:
            nloop = 1
        else:
            nloop = 1 + int(dp/pinc)
            dp = dp/float(nloop)

        for n in range(nloop):
            p1 = p2
            t1 = t2
            pi1 = pi2
            th1 = th2
            qv1 = qv2
            ql1 = ql2 
            qi1 = qi2
            thv1 = thv2

            p2 = p2-dp
            pi2 = np.power(p2*rp00, rddcp)
            thlast = th1
            i = 0
            not_converged = True

            while not_converged:
                i = i+1
                t2 = thlast * pi2
                if ice:
                    fliq = max(min((t2-233.15)/(273.15-233.15),1.0),0.0)
                    fice = 1.0 - fliq

                else:
                    fliq = 1.0
                    fice = 0.0
                qv2 = min(qt, fliq*getqvs(p2,t2)+fice*getqvi(p2,t2))
                qi2 = max(fice*(qt-qv2),0.0)
                ql2 = max(qt-qv2-qi2, 0.0)
                tbar = 0.5 * (t1+t2)
                qvbar = 0.5 * (qv1 + qv2)
                qlbar = 0.5 * (ql1+ql2)
                qibar = 0.5 * (qi1 + qi2)

                lhv = lv1 - lv2 * tbar
                lhs = ls1-ls2*tbar
                lhf = lhs - lhv

                rm = rd+rv*qvbar
                cpm = cp+cpv*qvbar+cpl*qlbar+cpi*qibar
                
                th2 = th1 * np.exp(lhv * (ql2-ql1)/(cpm*tbar)+
                                   lhs * (qi2-qi1)/(cpm*tbar)+
                                   (rm/cpm-rd/cp)*np.log(p2/p1))

                if th2>1000:
                    #print(n,k,i)
                    pass
                    #print(th2, th1, lhv, ql2, ql1, cpm, tbar, lhs, qi2, qi1, rm, 
                    #rd, cp, p2, p1, np.log(p2/p1))

                if i>90:
                    #print(i,th2,thlast, th2-thlast)
                    pass

                if i>100:
                    raise ArithmeticError("Lack of convergence, stopping iteration")
                #print(th2, thlast, converge)
                if abs(th2-thlast) > converge:
                    thlast = thlast+0.3*(th2-thlast)
                else:
                    not_converged = False

            if ql2 >= 1.0e-10:
                cloud = True

            if 'p' in adiabat:
                #pseudoadiabat
                qt = qv2
                ql2 = 0.0
                qi2 = 0.0

        thv2 = th2 * (1.0+reps*qv2)/(1.0+qv2+ql2+qi2)
        b2 = g* (thv2-thv[k])/thv[k]
        dz = -cpdg * 0.5 * (thv[k] + thv[k-1])*(pi[k] - pi[k-1])

        the = getthe(p2,t2,t2,qv2)

        #Get contributions to CAPE and CIN
        if b2 >= 0.0 and b1 < 0.0:
            #first trip into positive area.
            ps = p[k-1]+(p[k]-p[k-1]) * (0.0-b1)/(b2-b1)
            frac = b2/(b2-b1)
            parea = 0.5*b2*dz*frac
            narea = narea - 0.5 * b1*dz*(1.0-frac)
            cin = cin+narea
        elif b2<0.0 and b1>0.0:
            #first trip into negative area
            ps = p[k-1] + (p[k]-p[k-1])*(0.0-b1)/(b2-b1)
            frac = b1/(b1-b2)
            parea = 0.5*b1*dz*frac
            narea = -0.5*b2*dz*(1.0-frac)

        elif b2<0.0:
            #still collecting negative buoyancy
            parea = 0.0
            narea = narea-0.5*dz*(b1+b2)
        else:
            #still collecting positive buoyancy
            parea = 0.5*dz*(b1+b2)
            narea = 0.0

        #print(b1,b2, parea,narea)
        cape = cape + max(0.0, parea)
        if p[k] <= 10000.0 and b2 <0.0:
            #stop if b<0 and p<100mb
            doit = False        


    return cape, cin


# Additional Thermodynamic functions
def getqvs(p,t):
    '''
    #Calculate saturation from pressure and temperature
    '''
    eps = 287.04/461.5
    es = 611.2*np.exp(17.67*(t-273.15)/(t-29.65)) # Calculate vapor pressure
    return eps*es/(p-es)

def getqvi(p,t):
    '''
    #Calculate saturation over ice from pressure and temperature
    '''
    eps = 287.04/461.5
    es = 611.2*np.exp(21.8745584*(t-273.15)/(t-7.66)) # Calculate vapor pressure over ice
    return eps*es/(p-es)

def getthe(p,t,td,q):
    if td - t >=-0.1:
        tlcl = t
    else:
        tlcl = 56.0 + np.power(np.power((td-56.0),-1) + 0.00125* np.log(t/td),-1)

    return (t*( np.power((100000.0/p),(0.2854*(1.0-0.28*q))) )   
            *np.exp( ((3376.0/tlcl)-2.54)*q*(1.0+0.81*q) ))

In [4]:
def read_head(headfile,h5file):
    
    # Function that reads header files from RAMS
    
    # Inputs:
    #   headfile: header file including full path in str format
    #   h5file: h5 datafile including full path in str format
    
    # Returns:
    #   zmn: height levels for momentum values (i.e., grid box upper and lower levels)
    #   ztn: height levels for thermodynaic values (i.e., grid box centers)
    #   nx:: the number of x points for the domain associated with the h5file
    #   ny: the number of y points for the domain associated with the h5file
    #   npa: the number of surface patches
    
    
    dom_num = h5file[h5file.index('.h5')-1] # Find index of .h5 to determine position showing which nest domain to use

    with open(headfile) as f:
        contents = f.readlines()
        
    idx_zmn = contents.index('__zmn0'+dom_num+'\n')
    nz_m = int(contents[idx_zmn+1])
    zmn = np.zeros(nz_m)
    for i in np.arange(0,nz_m):
        zmn[i] =  float(contents[idx_zmn+2+i])
    
    idx_ztn = contents.index('__ztn0'+dom_num+'\n')
    nz_t = int(contents[idx_ztn+1])
    ztn = np.zeros(nz_t)
    for i in np.arange(0,nz_t):
        ztn[i] =  float(contents[idx_ztn+2+i])
    
    ztop = np.max(ztn) # Model domain top (m)
    
    # Grad the size of the horizontal grid spacing
    idx_dxy = contents.index('__deltaxn\n')
    dxy = float(contents[idx_dxy+1+int(dom_num)].strip())

    idx_npatch = contents.index('__npatch\n')
    npa = int(contents[idx_npatch+2])
    
    idx_ny = contents.index('__nnyp\n')
    idx_nx = contents.index('__nnxp\n')
    ny = np.ones(int(contents[idx_ny+1]))
    nx = np.ones(int(contents[idx_ny+1]))
    for i in np.arange(0,len(ny)):
        nx[i] = int(contents[idx_nx+2+i])
        ny[i] = int(contents[idx_ny+2+i])

    ny_out = ny[int(dom_num)-1]
    nx_out = nx[int(dom_num)-1]

    return zmn, ztn, nx_out, ny_out, dxy, npa 

In [5]:
mpl.rcParams.update({'font.size': 15})
lwid = 3.0

In [6]:
# Define INCUS simulations / cases
cases = ['ARG1.1-R_old','ARG1.2-R','BRA1.1-R','BRA1.2-R','AUS1.1-R','DRC1.1-R','PHI1.1-R','PHI2.1-R','WPO1.1-R','USA1.1-R','RSA1.1-R']

In [7]:
# Define Convective Encironment Parameter Variables
# ml = midlevels
# ll = lowlevls
# sfc = surface
#250, 500, 850 = hPa levels
#Wind Shear
shr_ml = OrderedDict()
shr_ll = OrderedDict()
#Relative humidity
rh_ml = OrderedDict()
rh_ll = OrderedDict()
rh_sfc = OrderedDict()
rh_850 = OrderedDict()
rh_500 = OrderedDict()
rh_250 = OrderedDict()
#Water vapor mixing ratio
rv_ml = OrderedDict()
rv_ll = OrderedDict()
rv_sfc = OrderedDict()
rv_850 = OrderedDict()
rv_500 = OrderedDict()
rv_250 = OrderedDict()
#Wind speed
spd_ml = OrderedDict()
spd_ll = OrderedDict()
spd_sfc = OrderedDict()
spd_850 = OrderedDict()
spd_500 = OrderedDict()
spd_250 = OrderedDict()
#Condensate Mixing Ratio
c_ml = OrderedDict()
c_ll = OrderedDict()
c_sfc = OrderedDict()
c_850 = OrderedDict()
c_500 = OrderedDict()
c_250 = OrderedDict()
#Vertical Velocity
w_ml = OrderedDict()
w_ll = OrderedDict()
w_sfc = OrderedDict()
w_850 = OrderedDict()
w_500 = OrderedDict()
w_250 = OrderedDict()
#Temperature
t_sfc = OrderedDict()
t_ll = OrderedDict()
t_ml = OrderedDict()
t_mllr = OrderedDict()
t_850 = OrderedDict()
t_500 = OrderedDict()
t_250 = OrderedDict()
# Other parameters
fl_p = OrderedDict() #Freezing Level
lcl_p = OrderedDict() #Lifting condensation level pressure
lcl_z = OrderedDict() #Lifting condensation level altitude
lcl_t = OrderedDict() #Lifting condensation level temperature
wcd_p = OrderedDict() #Warm Cloud depth in pressure
#Metpy CAPE and CIN
mucape = OrderedDict() #most unstable CAPE
mucin = OrderedDict() #most unstable CIN
mlcape = OrderedDict() #mixed layer CAPE
mlcin = OrderedDict() #mixed layer CIN
#CAPE and CIN using defined function above
mucapeGB = OrderedDict()
mucinGB = OrderedDict()
mlcapeGB = OrderedDict()
mlcinGB = OrderedDict()
mlcapeGB1 = OrderedDict()
mlcinGB1 = OrderedDict()

# Total column integrated variables
tcw = OrderedDict() #water
tcwv = OrderedDict() #water vapor
tcwc = OrderedDict() #condensate
tcl = OrderedDict() #liquid condensate
tci = OrderedDict() #unrimed ice condensate 
tcri = OrderedDict() #rimed ice condensate

# Define same variables but for the subgrids
shr_mls = OrderedDict()
shr_lls = OrderedDict()

rh_mls = OrderedDict()
rh_lls = OrderedDict()
rh_sfcs = OrderedDict()
rh_850s = OrderedDict()
rh_500s = OrderedDict()
rh_250s = OrderedDict()

rv_mls = OrderedDict()
rv_lls = OrderedDict()
rv_sfcs = OrderedDict()
rv_850s = OrderedDict()
rv_500s = OrderedDict()
rv_250s = OrderedDict()

spd_mls = OrderedDict()
spd_lls = OrderedDict()
spd_sfcs = OrderedDict()
spd_850s = OrderedDict()
spd_500s = OrderedDict()
spd_250s = OrderedDict()

t_sfcs = OrderedDict()
t_lls = OrderedDict()
t_mls = OrderedDict()
t_mllrs = OrderedDict()
t_850s = OrderedDict()
t_500s = OrderedDict()
t_250s = OrderedDict()

c_sfcs = OrderedDict()
c_lls = OrderedDict()
c_mls = OrderedDict()
c_850s = OrderedDict()
c_500s = OrderedDict()
c_250s = OrderedDict()

w_sfcs = OrderedDict()
w_lls = OrderedDict()
w_mls = OrderedDict()
w_850s = OrderedDict()
w_500s = OrderedDict()
w_250s = OrderedDict()


fl_ps = OrderedDict()
lcl_ps = OrderedDict()
lcl_zs = OrderedDict()
lcl_ts = OrderedDict()
wcd_ps = OrderedDict()

mucapes = OrderedDict()
mucins = OrderedDict()
mlcapes = OrderedDict()
mlcins = OrderedDict()

tcws = OrderedDict()
tcwvs = OrderedDict()
tcwcs = OrderedDict()
tcls = OrderedDict()
tcis = OrderedDict()
tcris = OrderedDict()

# Define variables for analysis
spacings = [3,4,5,9,27] # 9km, 7km, 5km, 3km, 1km
#spacings = [3,4,5,9,27,54] # 9km, 7km, 5km, 3km, 1km, 500m
pmllvls = [800,400] # Pressure levels for mid-levels
plllvls = [1000,800] # Pressure levels for low-levels
save_add = 'all_100'

# Loop through simulations
for c in np.arange(0,len(cases)):
    cn = cases[c]
    print(cn)
    # Grab model files associated with each simulations/case
    files = sorted(glob.glob('/monsoon/MODEL/LES_MODEL_DATA/V0/'+cn+'-V0/G3/out_30s/a-A*g3.h5'))
    hfiles = sorted(glob.glob('/monsoon/MODEL/LES_MODEL_DATA/V0/'+cn+'-V0/G3/out_30s/a-A*head.txt'))
    zmn, ztn, nx_out, ny_out, dxy, npa = read_head(hfiles[0],files[0])

    # Define paths for input data (saved profiles) and output data
    infilename = '/tempest/pmarin/monsoon/ENV/Sim_Profs_range_'+save_add+'_'+cn+'.p'
    savepath = '/tempest/pmarin/monsoon/ENV/'+cn+'/'+save_add+'/'
    plotpath = '/tempest/pmarin/monsoon/ENV/Plots/'+cn+'/'
    if not os.path.exists(plotpath):
        os.makedirs(plotpath)
    if not os.path.exists(savepath):
        os.makedirs(savepath)

    # Grab saved profiles (saved from reanal_dom_subdom_prof code)
    with open(infilename, 'rb') as f:
        [profd,latlon,numg,profds,latlons,latlons_id,numgs] = pickle.load(f)    
        num_prof = numg[cn]
        print(num_prof)

    # Defining convective environment parameter variables
    shr_ml[cn] = np.zeros(num_prof)
    shr_ll[cn] = np.zeros(num_prof)

    rh_sfc[cn] = np.zeros(num_prof)
    rh_ml[cn] = np.zeros(num_prof)
    rh_ll[cn] = np.zeros(num_prof)
    rh_850[cn] = np.zeros(num_prof)
    rh_500[cn] = np.zeros(num_prof)
    rh_250[cn] = np.zeros(num_prof)

    rv_sfc[cn] = np.zeros(num_prof)
    rv_ml[cn] = np.zeros(num_prof)
    rv_ll[cn] = np.zeros(num_prof)
    rv_850[cn] = np.zeros(num_prof)
    rv_500[cn] = np.zeros(num_prof)
    rv_250[cn] = np.zeros(num_prof)
    
    spd_sfc[cn] = np.zeros(num_prof)
    spd_ml[cn] = np.zeros(num_prof)
    spd_ll[cn] = np.zeros(num_prof)
    spd_850[cn] = np.zeros(num_prof)
    spd_500[cn] = np.zeros(num_prof)
    spd_250[cn] = np.zeros(num_prof)

    t_sfc[cn] = np.zeros(num_prof)
    t_ml[cn] = np.zeros(num_prof)
    t_mllr[cn] = np.zeros(num_prof)
    t_ll[cn] = np.zeros(num_prof)
    t_850[cn] = np.zeros(num_prof)
    t_500[cn] = np.zeros(num_prof)
    t_250[cn] = np.zeros(num_prof)
    
    c_sfc[cn] = np.zeros(num_prof)
    c_850[cn] = np.zeros(num_prof)
    c_500[cn] = np.zeros(num_prof)
    c_250[cn] = np.zeros(num_prof)
    c_ml[cn] = np.zeros(num_prof)
    c_ll[cn] = np.zeros(num_prof)

    w_sfc[cn] = np.zeros(num_prof)
    w_850[cn] = np.zeros(num_prof)
    w_500[cn] = np.zeros(num_prof)
    w_250[cn] = np.zeros(num_prof)
    w_ml[cn] = np.zeros(num_prof)
    w_ll[cn] = np.zeros(num_prof)
    
    fl_p[cn] = np.zeros(num_prof)
    lcl_t[cn] = np.zeros(num_prof)
    lcl_p[cn] = np.zeros(num_prof)
    lcl_z[cn] = np.zeros(num_prof)
    wcd_p[cn] = np.zeros(num_prof)
    mucape[cn] = np.zeros(num_prof)
    mucin[cn] = np.zeros(num_prof)
    mlcape[cn] = np.zeros(num_prof)
    mlcin[cn] = np.zeros(num_prof)
    mucapeGB[cn] = np.zeros(num_prof)
    mucinGB[cn] = np.zeros(num_prof)
    mlcapeGB[cn] = np.zeros(num_prof)
    mlcinGB[cn] = np.zeros(num_prof)
    mlcapeGB1[cn] = np.zeros(num_prof)
    mlcinGB1[cn] = np.zeros(num_prof)
    tcwv[cn] = np.zeros(num_prof)
    tcwc[cn] = np.zeros(num_prof)
    tcw[cn] = np.zeros(num_prof)
    tcl[cn] = np.zeros(num_prof)
    tci[cn] = np.zeros(num_prof)
    tcri[cn] = np.zeros(num_prof)
    
    # Loop through each profile from the saved profiles for the Reanalysis-Size Grids (i.e., 27KM_AREAS)
    # Calculate and save convective environment parameters
    for i in np.arange(0,num_prof):
        print(i)
            
        temp = profd[cn]['temp'][i]
        pres = profd[cn]['pres'][i]/100
        rv = profd[cn]['RV'][i]
        u = profd[cn]['UP'][i]
        v = profd[cn]['VP'][i]       
        w = profd[cn]['WP'][i]       
        rtc = profd[cn]['RTC'][i]
        topt = profd[cn]['topt'][i]
        rl = profd[cn]['RTL'][i]
        ri = profd[cn]['RTI'][i]
        rri = profd[cn]['RTRI'][i]
        spd = np.sqrt(u*u+v*v)
        
        #print(pres[0])
        # Find indicies of certain pressure levels in model data
        idsfc = 1
        id850 = np.where(np.abs(pres-850) == np.min(np.abs(pres-850)))[0][0]
        id500 = np.where(np.abs(pres-500) == np.min(np.abs(pres-500)))[0][0]
        id250 = np.where(np.abs(pres-250) == np.min(np.abs(pres-250)))[0][0]

        # Save specific pressure level data
        t_sfc[cn][i] = temp[1]
        rv_sfc[cn][i] = rv[1]
        spd_sfc[cn][i] = spd[1]
        c_sfc[cn][i] = rtc[1]
        w_sfc[cn][i] = w[1]
        
        t_850[cn][i] = temp[id850]
        rv_850[cn][i] = rv[id850]
        spd_850[cn][i] = spd[id850]
        c_850[cn][i] = rtc[id850]       
        w_850[cn][i] = w[id850]       
        
        t_500[cn][i] = temp[id500]
        rv_500[cn][i] = rv[id500]
        spd_500[cn][i] = spd[id500]
        c_500[cn][i] = rtc[id500]
        w_500[cn][i] = w[id500]

        t_250[cn][i] = temp[id250]
        rv_250[cn][i] = rv[id250]
        spd_250[cn][i] = spd[id250]
        c_250[cn][i] = rtc[id250]
        w_250[cn][i] = w[id250]

        t_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,temp)
        rv_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,rv)
        spd_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,spd)
        c_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,rtc)
        w_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,w)

        t_mllr[cn][i] = calc_lapse_rate_p(plllvls,pres,temp,zmn)
        t_ll[cn][i] = calc_mean_layer_p(plllvls,pres,temp)
        rv_ll[cn][i] = calc_mean_layer_p(plllvls,pres,rv)
        spd_ll[cn][i] = calc_mean_layer_p(plllvls,pres,spd)                
        c_ll[cn][i] = calc_mean_layer_p(plllvls,pres,rtc)
        w_ll[cn][i] = calc_mean_layer_p(plllvls,pres,w)
                
        tcwv[cn][i] = calc_tcwv(pres*100,temp,rv,zmn,topt)
        tcwc[cn][i] = calc_tcwv(pres*100,temp,rtc,zmn,topt)
        tcw[cn][i] = calc_tcwv(pres*100,temp,rtc+rv,zmn,topt)
        tcl[cn][i] = calc_tcwv(pres*100,temp,rl,zmn,topt)
        tci[cn][i] = calc_tcwv(pres*100,temp,ri,zmn,topt)
        tcri[cn][i] = calc_tcwv(pres*100,temp,rri,zmn,topt)

        id_0c = np.where(np.abs(temp-273.15) == np.min(np.abs((temp-273.15))))[0]
        fl_p[cn][i] = pres[int(id_0c[0])]
        
        # Save temperature and shear data
        shr_ml[cn][i] = calc_layer_shear_p(pmllvls,pres,u,v)
        shr_ll[cn][i] = calc_layer_shear_p(plllvls,pres,u,v)        
        
        # convert profile to variables / units for metpy
        p_mp = pres*units.hPa

        t_mp = temp-273.15
        t_mp = t_mp*units.degC

        # Calculate vapor pressure
        mixing = rv*units('kg/kg')
        e_mp = mpcalc.vapor_pressure(p_mp, mixing)
        e_mp[e_mp==0]=np.NaN
        e_mp[np.isnan(e_mp)]=np.nanmin(e_mp)

        es_mp = mpcalc.saturation_vapor_pressure(t_mp).to('hPa')
        rh = (e_mp/es_mp).magnitude
#        print(rh)

        # Save rh data
        rh_ml[cn][i] = calc_mean_layer_p(pmllvls,pres,rh)
        rh_ll[cn][i] = calc_mean_layer_p(plllvls,pres,rh)
        rh_sfc[cn][i] = rh[1]     
        rh_850[cn][i] = rh[id850]     
        rh_500[cn][i] = rh[id500]     
        rh_250[cn][i] = rh[id250]     
        
        # Caclculate depoint temperature
        td_mp = mpcalc.dewpoint(e_mp)

        # Convert winds from m/s to knots
        u_mp = u*1.94384*units.knots
        v_mp = v*1.94384*units.knots

        fig = plt.figure(figsize=(9, 9))
        ##add_metpy_logo(fig, 115, 100)
        skew = SkewT(fig, rotation=45)

        # Plot the data using normal plotting functions, in this case using
        # log scaling in Y, as dictated by the typical meteorological plot.
        skew.plot(p_mp, t_mp, 'r',linewidth=lwid)
        skew.plot(p_mp, td_mp, 'g',linewidth=lwid)
        ##skew.plot_barbs(p, u, v)
        skew.ax.set_ylim(1000, 100)
        skew.ax.set_xlim(-30, 40)

        # Calculate LCL height and plot as black dot. Because `p`'s first value is
        # ~1000 mb and its last value is ~250 mb, the `0` index is selected for
        # `p`, `T`, and `Td` to lift the parcel from the surface. If `p` was inverted,
        # i.e. start from low value, 250 mb, to a high value, 1000 mb, the `-1` index
        # should be selected.
        lcl_pressure, lcl_temperature = mpcalc.lcl(p_mp[0], t_mp[0], td_mp[0])
        lcl_p[cn][i] = lcl_pressure.magnitude
        lcl_t[cn][i] = lcl_temperature.magnitude
        skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black',linewidth=lwid)

        wcd_p[cn][i] = lcl_p[cn][i]-fl_p[cn][i]
              
        # Calculate full parcel profile and add to plot as black line
        prof = mpcalc.parcel_profile(p_mp, t_mp[0], td_mp[0]).to('degC')
        skew.plot(p_mp, prof, 'k',linewidth=lwid)
        cape, cin = mpcalc.cape_cin(p_mp, t_mp, td_mp, prof)
        p_mlcape, p_mlcin = mpcalc.mixed_layer_cape_cin(p_mp, t_mp, td_mp)
        p_mucape, p_mucin = mpcalc.most_unstable_cape_cin(p_mp, t_mp, td_mp)

        mlcape[cn][i] = p_mlcape.magnitude
        mlcin[cn][i] = p_mlcin.magnitude
        mucape[cn][i] = p_mucape.magnitude
        mucin[cn][i] = p_mucin.magnitude
        
        mlcapeGB[cn][i], mlcinGB[cn][i] = getcape(p_mp.magnitude, t_mp.magnitude, td_mp.magnitude, source_parc = 'ml', ml_depth = 500.0, adiabat = 'pl')
        mucapeGB[cn][i], mucinGB[cn][i] = getcape(p_mp.magnitude, t_mp.magnitude, td_mp.magnitude, source_parc = 'mu', ml_depth = 500.0, adiabat = 'pl')
        mlcapeGB1[cn][i], mlcinGB1[cn][i] = getcape(p_mp.magnitude, t_mp.magnitude, td_mp.magnitude, source_parc = 'ml', ml_depth = 1000.0, adiabat = 'pl')
                
        # Shade areas of CAPE and CIN
        #skew.shade_cin(p_mp, t_mp, prof, td_mp)
        #skew.shade_cape(p_mp, t_mp, prof)

        # An example of a slanted line at constant T -- in this case the 0
        # isotherm
        skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

        # Add the relevant special lines
        skew.plot_dry_adiabats()
        skew.plot_moist_adiabats()
        skew.plot_mixing_lines()

        plt.xlabel('Temperature (C)')
        plt.ylabel('Pressure (hPa)')

        plt.title(cn+': Dom'+str(i)+' | CAPE:'+str(int(p_mlcape.magnitude))+' CIN:'+str(int(p_mlcin.magnitude)))
        # Show the plot
#        plt.show()   
        plt.tight_layout()
        fig.savefig(plotpath+'SkewT_LogP_'+cn+str(i)+'_MLCAPE.png')    
        plt.close(fig)
   
    # Created a dictionary for saving all the variables
    save_vars = OrderedDict()
    save_vars['shr_mls'] = shr_ml
    save_vars['shr_lls'] = shr_ll

    save_vars['rh_mls'] = rh_ml
    save_vars['rh_lls'] = rh_ll
    save_vars['rh_sfcs'] = rh_sfc
    save_vars['rh_850s'] = rh_850
    save_vars['rh_500s'] = rh_500
    save_vars['rh_250s'] = rh_250

    save_vars['rv_mls'] = rv_ml
    save_vars['rv_lls'] = rv_ll
    save_vars['rv_sfcs'] = rv_sfc
    save_vars['rv_850s'] = rv_850
    save_vars['rv_500s'] = rv_500
    save_vars['rv_250s'] = rv_250

    save_vars['t_mls'] = t_ml
    save_vars['t_mllrs'] = t_mllr
    save_vars['t_lls'] = t_ll
    save_vars['t_sfcs'] = t_sfc
    save_vars['t_850s'] = t_850
    save_vars['t_500s'] = t_500
    save_vars['t_250s'] = t_250                

    save_vars['spd_mls'] = spd_ml
    save_vars['spd_lls'] = spd_ll
    save_vars['spd_sfcs'] = spd_sfc
    save_vars['spd_850s'] = spd_850
    save_vars['spd_500s'] = spd_500
    save_vars['spd_250s'] = spd_250                                

    save_vars['c_mls'] = c_ml
    save_vars['c_lls'] = c_ll
    save_vars['c_sfcs'] = c_sfc
    save_vars['c_850s'] = c_850
    save_vars['c_500s'] = c_500
    save_vars['c_250s'] = c_250                                

    save_vars['w_mls'] = w_ml
    save_vars['w_lls'] = w_ll
    save_vars['w_sfcs'] = w_sfc
    save_vars['w_850s'] = w_850
    save_vars['w_500s'] = w_500
    save_vars['w_250s'] = w_250                                
    
    save_vars['rh_mls'] = rh_ml
    save_vars['rh_lls'] = rh_ll
    save_vars['rh_sfcs'] = rh_sfc
    save_vars['rh_850s'] = rh_850
    save_vars['rh_500s'] = rh_500
    save_vars['rh_250s'] = rh_250
                
    save_vars['fl_ps'] = fl_p
    save_vars['lcl_ts'] = lcl_t
    save_vars['lcl_ps'] = lcl_p
    save_vars['lcl_zs'] = lcl_z
    save_vars['wcd_ps'] = wcd_p
    save_vars['t_sfcs'] = t_sfc
    save_vars['mucapes'] = mucape
    save_vars['mucins'] = mucin
    save_vars['mlcapes'] = mlcape
    save_vars['mlcins'] = mlcin
    save_vars['mucapesGB'] = mucapeGB
    save_vars['mucinsGB'] = mucinGB
    save_vars['mlcapesGB'] = mlcapeGB
    save_vars['mlcinsGB'] = mlcinGB
    save_vars['mlcapesGB1'] = mlcapeGB1
    save_vars['mlcinsGB1'] = mlcinGB1
    save_vars['tcwvs'] = tcwv
    save_vars['tcwcs'] = tcwc
    save_vars['tcws'] = tcw
    save_vars['tcls'] = tcl
    save_vars['tcis'] = tci
    save_vars['tcris'] = tcri

    # Save convective environment parameters for Reanalysis-Sized Domains
    #savefilename = cn+'_Environments_ERA5_'+save_add+'.p'
    #with open(savepath+savefilename, "wb") as output_file:
    #    pickle.dump(save_vars, output_file)

    # Start calculations for subdomains
    subs = numg[cn]
    # Loop through each profile from the saved profiles for the Reanalysis Subdomains (i.e., 1KM_SUBGRIDS)
    # Calculate and save convective environment parameters
    for s in np.arange(0,subs):
        print(s)
        # Create filename for each ERA5 subdomain
        savefilename = cn+'_Environments_SubERA5_'+save_add+'_'+str(s)+'.p'

        # Loop through and test several subgrid spacings
        for gs in np.arange(0,len(spacings)):

            ss = spacings[gs]

            num_prof = numgs[cn,s,gs] # number of subgrids with era5 subdomain
            print(num_prof)

            #Define convective environment parameter arrays for each case / subgrid size
            shr_mls[cn,gs] = np.zeros(num_prof)
            shr_lls[cn,gs] = np.zeros(num_prof)

            rh_sfcs[cn,gs] = np.zeros(num_prof)
            rh_mls[cn,gs] = np.zeros(num_prof)
            rh_lls[cn,gs] = np.zeros(num_prof)
            rh_850s[cn,gs] = np.zeros(num_prof)
            rh_500s[cn,gs] = np.zeros(num_prof)
            rh_250s[cn,gs] = np.zeros(num_prof)

            rv_sfcs[cn,gs] = np.zeros(num_prof)
            rv_mls[cn,gs] = np.zeros(num_prof)
            rv_lls[cn,gs] = np.zeros(num_prof)
            rv_850s[cn,gs] = np.zeros(num_prof)
            rv_500s[cn,gs] = np.zeros(num_prof)
            rv_250s[cn,gs] = np.zeros(num_prof)                

            spd_sfcs[cn,gs] = np.zeros(num_prof)
            spd_mls[cn,gs] = np.zeros(num_prof)
            spd_lls[cn,gs] = np.zeros(num_prof)
            spd_850s[cn,gs] = np.zeros(num_prof)
            spd_500s[cn,gs] = np.zeros(num_prof)
            spd_250s[cn,gs] = np.zeros(num_prof)                

            c_sfcs[cn,gs] = np.zeros(num_prof)
            c_mls[cn,gs] = np.zeros(num_prof)
            c_lls[cn,gs] = np.zeros(num_prof)
            c_850s[cn,gs] = np.zeros(num_prof)
            c_500s[cn,gs] = np.zeros(num_prof)
            c_250s[cn,gs] = np.zeros(num_prof)                

            w_sfcs[cn,gs] = np.zeros(num_prof)
            w_mls[cn,gs] = np.zeros(num_prof)
            w_lls[cn,gs] = np.zeros(num_prof)
            w_850s[cn,gs] = np.zeros(num_prof)
            w_500s[cn,gs] = np.zeros(num_prof)
            w_250s[cn,gs] = np.zeros(num_prof)                
            
            t_sfcs[cn,gs] = np.zeros(num_prof)
            t_mls[cn,gs] = np.zeros(num_prof)
            t_mllrs[cn,gs] = np.zeros(num_prof)
            t_lls[cn,gs] = np.zeros(num_prof)
            t_850s[cn,gs] = np.zeros(num_prof)
            t_500s[cn,gs] = np.zeros(num_prof)
            t_250s[cn,gs] = np.zeros(num_prof)                

            fl_ps[cn,gs] = np.zeros(num_prof)
            lcl_ts[cn,gs] = np.zeros(num_prof)
            lcl_ps[cn,gs] = np.zeros(num_prof)
            lcl_zs[cn,gs] = np.zeros(num_prof)
            wcd_ps[cn,gs] = np.zeros(num_prof)
            mucapes[cn,gs] = np.zeros(num_prof)
            mucins[cn,gs] = np.zeros(num_prof)
            mlcapes[cn,gs] = np.zeros(num_prof)
            mlcins[cn,gs] = np.zeros(num_prof)
            mucapesGB[cn,gs] = np.zeros(num_prof)
            mucinsGB[cn,gs] = np.zeros(num_prof)
            mlcapesGB[cn,gs] = np.zeros(num_prof)
            mlcinsGB[cn,gs] = np.zeros(num_prof)
            mlcapesGB1[cn,gs] = np.zeros(num_prof)
            mlcinsGB1[cn,gs] = np.zeros(num_prof)
            tcwvs[cn,gs] = np.zeros(num_prof)
            tcwcs[cn,gs] = np.zeros(num_prof)
            tcws[cn,gs] = np.zeros(num_prof)
            tcls[cn,gs] = np.zeros(num_prof)
            tcis[cn,gs] = np.zeros(num_prof)
            tcris[cn,gs] = np.zeros(num_prof)

            #Loop through subdomains and calculate convective environment parameters
            for i in np.arange(0,num_prof):
                temp = profds[cn]['temp'][s][ss][i]
                pres = profds[cn]['pres'][s][ss][i]/100
                rv = profds[cn]['RV'][s][ss][i]
                u = profds[cn]['UP'][s][ss][i]
                v = profds[cn]['VP'][s][ss][i]       
                w = profds[cn]['WP'][s][ss][i]       
                rtc = profds[cn]['RTC'][s][ss][i]
                topt = profds[cn]['topt'][s][ss][i]
                rl = profds[cn]['RTL'][s][ss][i]
                ri = profds[cn]['RTI'][s][ss][i]
                rri = profds[cn]['RTRI'][s][ss][i]
                spd = np.sqrt(u*u+v*v)

                #print(pres[0])
                idsfc = 1
                id850 = np.where(np.abs(pres-850) == np.min(np.abs(pres-850)))[0][0]
                id500 = np.where(np.abs(pres-500) == np.min(np.abs(pres-500)))[0][0]
                id250 = np.where(np.abs(pres-250) == np.min(np.abs(pres-250)))[0][0]

                t_sfcs[cn,gs][i] = temp[1]
                rv_sfcs[cn,gs][i] = rv[1]
                spd_sfcs[cn,gs][i] = spd[1]
                c_sfcs[cn,gs][i] = rtc[1]
                w_sfcs[cn,gs][i] = w[1]

                t_850s[cn,gs][i] = temp[id850]
                rv_850s[cn,gs][i] = rv[id850]
                spd_850s[cn,gs][i] = spd[id850]
                c_850s[cn,gs][i] = rtc[id850]
                w_850s[cn,gs][i] = w[id850]

                t_500s[cn,gs][i] = temp[id500]
                rv_500s[cn,gs][i] = rv[id500]
                spd_500s[cn,gs][i] = spd[id500]
                c_500s[cn,gs][i] = rtc[id500]
                w_500s[cn,gs][i] = w[id500]
                
                t_250s[cn,gs][i] = temp[id250]
                rv_250s[cn,gs][i] = rv[id250]
                spd_250s[cn,gs][i] = spd[id250]
                c_250s[cn,gs][i] = rtc[id250]
                w_250s[cn,gs][i] = w[id250]

                t_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,temp)
                t_mllrs[cn,gs][i] = calc_lapse_rate_p(pmllvls,pres,temp,zmn)
                t_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,temp)

                c_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,rtc)
                c_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,rtc)

                w_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,w)
                w_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,w)

                rv_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,rv)
                rv_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,rv)
                
                spd_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,spd)
                spd_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,spd)
                
                tcwvs[cn,gs][i] = calc_tcwv(pres*100,temp,rv,zmn,topt)               
                tcwcs[cn,gs][i] = calc_tcwv(pres*100,temp,rtc,zmn,topt)               
                tcws[cn,gs][i] = calc_tcwv(pres*100,temp,rtc+rv,zmn,topt)               
                tcls[cn,gs][i] = calc_tcwv(pres*100,temp,rl,zmn,topt)               
                tcis[cn,gs][i] = calc_tcwv(pres*100,temp,ri,zmn,topt)               
                tcris[cn,gs][i] = calc_tcwv(pres*100,temp,rri,zmn,topt)               
                
                id_0c = np.where(np.abs(temp-273.15) == np.min(np.abs((temp-273.15))))[0]
                fl_ps[cn,gs][i] = pres[int(id_0c[0])]

                # Save temperature and shear data
                shr_mls[cn,gs][i] = calc_layer_shear_p(pmllvls,pres,u,v)
                shr_lls[cn,gs][i] = calc_layer_shear_p(plllvls,pres,u,v)        

                # convert profile to variables / units for metpy
                p_mp = pres*units.hPa

                t_mp = temp-273.15
                t_mp = t_mp*units.degC

                # Calculate vapor pressure
                mixing = rv*units('kg/kg')
                e_mp = mpcalc.vapor_pressure(p_mp, mixing)
                e_mp[e_mp==0]=np.NaN
                e_mp[np.isnan(e_mp)]=np.nanmin(e_mp)

                es_mp = mpcalc.saturation_vapor_pressure(t_mp).to('hPa')
                rh = (e_mp/es_mp).magnitude
        #        print(rh)

                # Save rh data
                rh_mls[cn,gs][i] = calc_mean_layer_p(pmllvls,pres,rh)
                rh_lls[cn,gs][i] = calc_mean_layer_p(plllvls,pres,rh)
                rh_sfcs[cn,gs][i] = rh[1]     
                rh_850s[cn,gs][i] = rh[id850]
                rh_500s[cn,gs][i] = rh[id500]
                rh_250s[cn,gs][i] = rh[id250]

                # Caclculate depoint temperature
                td_mp = mpcalc.dewpoint(e_mp)

                # Convert winds from m/s to knots
                u_mp = u*1.94384*units.knots
                v_mp = v*1.94384*units.knots

                #fig = plt.figure(figsize=(9, 9))
                #skew = SkewT(fig, rotation=45)
                                
                # Plot the data using normal plotting functions, in this case using
                # log scaling in Y, as dictated by the typical meteorological plot.
                #skew.plot(p_mp, t_mp, 'r',linewidth=lwid)
                #skew.plot(p_mp, td_mp, 'g',linewidth=lwid)
                ##skew.plot_barbs(p, u, v)
                #skew.ax.set_ylim(1000, 100)
                #skew.ax.set_xlim(-30, 40)

                # Calculate LCL height and plot as black dot. Because `p`'s first value is
                # ~1000 mb and its last value is ~250 mb, the `0` index is selected for
                # `p`, `T`, and `Td` to lift the parcel from the surface. If `p` was inverted,
                # i.e. start from low value, 250 mb, to a high value, 1000 mb, the `-1` index
                # should be selected.
                lcl_pressure, lcl_temperature = mpcalc.lcl(p_mp[0], t_mp[0], td_mp[0])
                lcl_ps[cn,gs][i] = lcl_pressure.magnitude
                lcl_ts[cn,gs][i] = lcl_temperature.magnitude
                #skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black',linewidth=lwid)

                wcd_ps[cn,gs][i] = lcl_ps[cn,gs][i]-fl_ps[cn,gs][i]

                # Calculate full parcel profile and add to plot as black line
                prof = mpcalc.parcel_profile(p_mp, t_mp[0], td_mp[0]).to('degC')
                #skew.plot(p_mp, prof, 'k',linewidth=lwid)
                cape, cin = mpcalc.cape_cin(p_mp, t_mp, td_mp, prof)
                p_mlcape, p_mlcin = mpcalc.mixed_layer_cape_cin(p_mp, t_mp, td_mp)
                p_mucape, p_mucin = mpcalc.most_unstable_cape_cin(p_mp, t_mp, td_mp)

                mlcapes[cn,gs][i] = p_mlcape.magnitude
                mlcins[cn,gs][i] = p_mlcin.magnitude
                mucapes[cn,gs][i] = p_mucape.magnitude
                mucins[cn,gs][i] = p_mucin.magnitude
                
                mlcapesGB[cn,gs][i], mlcinsGB[cn,gs][i] = getcape(p_mp.magnitude, t_mp.magnitude, td_mp.magnitude, source_parc = 'ml', ml_depth = 500.0, adiabat = 'pl')
                mucapesGB[cn,gs][i], mucinsGB[cn,gs][i] = getcape(p_mp.magnitude, t_mp.magnitude, td_mp.magnitude, source_parc = 'mu', ml_depth = 500.0, adiabat = 'pl')
                mlcapesGB1[cn,gs][i], mlcinsGB1[cn,gs][i] = getcape(p_mp.magnitude, t_mp.magnitude, td_mp.magnitude, source_parc = 'ml', ml_depth = 1000.0, adiabat = 'pl')
                                
                # Shade areas of CAPE and CIN
                #skew.shade_cin(p_mp, t_mp, prof, td_mp)
                #skew.shade_cape(p_mp, t_mp, prof)

                # An example of a slanted line at constant T -- in this case the 0
                # isotherm
                #skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

                # Add the relevant special lines
                #skew.plot_dry_adiabats()
                #skew.plot_moist_adiabats()
                #skew.plot_mixing_lines()

                #plt.xlabel('Temperature (C)')
                #plt.ylabel('Pressure (hPa)')

                #plt.title(cn+': Dom'+str(i)+' | CAPE:'+str(int(p_mlcape.magnitude))+' CIN:'+str(int(p_mlcin.magnitude)))
                # Show the plot
        #        plt.show()   
                #plt.tight_layout()
                #fig.savefig(plotpath+'SkewT_LogP_'+cn+str(i)+'_MLCAPE'+str(s)+'_'+str(gs)+'.png')    
                #plt.close(fig)

            # Create a dictionary for saving subgrid convective environment parameters
            save_vars = OrderedDict()
            save_vars['shr_mls'] = shr_mls
            save_vars['shr_lls'] = shr_lls

            save_vars['rh_mls'] = rh_mls
            save_vars['rh_lls'] = rh_lls
            save_vars['rh_sfcs'] = rh_sfcs
            save_vars['rh_850s'] = rh_850s
            save_vars['rh_500s'] = rh_500s
            save_vars['rh_250s'] = rh_250s

            save_vars['rv_mls'] = rv_mls
            save_vars['rv_lls'] = rv_lls
            save_vars['rv_sfcs'] = rv_sfcs
            save_vars['rv_850s'] = rv_850s
            save_vars['rv_500s'] = rv_500s
            save_vars['rv_250s'] = rv_250s

            save_vars['t_mls'] = t_mls
            save_vars['t_mllrs'] = t_mllrs
            save_vars['t_lls'] = t_lls
            save_vars['t_sfcs'] = t_sfcs
            save_vars['t_850s'] = t_850s
            save_vars['t_500s'] = t_500s
            save_vars['t_250s'] = t_250s               

            save_vars['c_mls'] = c_mls
            save_vars['c_lls'] = c_lls
            save_vars['c_sfcs'] = c_sfcs
            save_vars['c_850s'] = c_850s
            save_vars['c_500s'] = c_500s
            save_vars['c_250s'] = c_250s               
            
            save_vars['w_mls'] = w_mls
            save_vars['w_lls'] = w_lls
            save_vars['w_sfcs'] = w_sfcs
            save_vars['w_850s'] = w_850s
            save_vars['w_500s'] = w_500s
            save_vars['w_250s'] = w_250s               
            
            save_vars['spd_mls'] = spd_mls
            save_vars['spd_lls'] = spd_lls
            save_vars['spd_sfcs'] = spd_sfcs
            save_vars['spd_850s'] = spd_850s
            save_vars['spd_500s'] = spd_500s
            save_vars['spd_250s'] = spd_250s                               

            save_vars['rh_mls'] = rh_mls
            save_vars['rh_lls'] = rh_lls
            save_vars['rh_sfcs'] = rh_sfcs
            save_vars['rh_850s'] = rh_850s
            save_vars['rh_500s'] = rh_500s
            save_vars['rh_250s'] = rh_250s               
                
            save_vars['fl_ps'] = fl_ps
            save_vars['lcl_ts'] = lcl_ts
            save_vars['lcl_ps'] = lcl_ps
            save_vars['lcl_zs'] = lcl_zs
            save_vars['wcd_ps'] = wcd_ps
            save_vars['mucapes'] = mucapes
            save_vars['mucins'] = mucins
            save_vars['mlcapes'] = mlcapes
            save_vars['mlcins'] = mlcins
            save_vars['mucapesGB'] = mucapesGB
            save_vars['mucinsGB'] = mucinsGB
            save_vars['mlcapesGB'] = mlcapesGB
            save_vars['mlcinsGB'] = mlcinsGB
            save_vars['mlcapesGB1'] = mlcapesGB1
            save_vars['mlcinsGB1'] = mlcinsGB1
            save_vars['tcwvs'] = tcwvs
            save_vars['tcwcs'] = tcwvs
            save_vars['tcws'] = tcwvs
            save_vars['tcls'] = tcls
            save_vars['tcis'] = tcis
            save_vars['tcris'] = tcris

        # Save convective environment parameters
        #with open(savepath+savefilename, "wb") as output_file:
        #    pickle.dump(save_vars, output_file)
        
        

ARG1.1-R_old
42
0


/tmp/ipykernel_1764305/3597088974.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  var1 = data[int(alt0[0]):int(alt1[0])]
/tmp/ipykernel_1764305/3597088974.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  var1 = temp[int(alt0[0])]
/tmp/ipykernel_1764305/3597088974.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  var2 = temp[int(alt1[0])]
/tmp/ipykernel_1764305/3597088974.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is depre

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0
9


NameError: name 'mucapesGB' is not defined

In [ ]:
print('Finished Analysis')

In [33]:
#Print out CAPE and CIN values
for i in np.arange(0,34):
    print(mlcapeGB1[cn][i], mlcinGB1[cn][i])
    print(mlcapeGB[cn][i], mlcinGB[cn][i])
    print(mlcape[cn][i], mlcin[cn][i])
    print('------')

107.05207156788754 236.64001491963023
82.09947632254976 269.79827222974967
34.872760457826566 -314.0260941178811
------
38.21947841982211 0.0
10.951437677488384 0.0
0.0 0.0
------
991.563569588227 47.03283525848084
973.9800512651274 63.963237087803016
892.3862916853253 -87.37384604841853
------
28.989696911271643 0.0
8.976662270309284 0.0
0.0 0.0
------
53.02936214967014 708.3423580593751
16.253304075974036 802.6205766699944
3.5927441286546595 -790.2663502549116
------
1607.775763681897 26.81660093087358
1629.902932105591 35.87219548009412
1488.3527424571137 -51.65489373206753
------
1414.1834211049365 20.203214077714065
1655.079169640341 23.54295510306521
1363.9351991738736 -39.487231100548144
------
2311.464798037349 27.91889213020775
2355.6599966267504 42.819300656690885
2137.5033001008774 -44.576631380596424
------
1570.2498734401167 118.6713592456509
2163.2879208283566 97.70348654583096
1523.6760170135706 -202.96384925730163
------
1243.0082742377388 122.78867271106694
1333.244940

In [34]:
for i in np.arange(0,34):
    print(mucapeGB[cn][i], mucinGB[cn][i])
    print(mucape[cn][i], mucin[cn][i])
    print('-----')

116.90508779629235 213.75047170041998
62.59688373075546 -267.0130208669855
-----
0.0 0.0
0.0 0.0
-----
1128.7202135043647 54.351339739677215
1030.1663092313984 -64.68608151839894
-----
131.75126919100384 630.0454927277453
100.20282163383999 -719.1707194937868
-----
753.8887435195563 117.22109861531831
649.581982292368 -161.01686504160355
-----
1873.2498494447498 30.77098343283641
1784.9907687844154 -37.72324483989059
-----
2096.529853304204 21.605786271085215
2040.22067221401 -30.294300364371423
-----
2545.8123662036246 57.46917216788174
2394.734436631226 -67.08876497003389
-----
2551.5338732591963 97.9554618181667
2428.9629534379333 -166.3474917728411
-----
1832.805558805072 67.26041012283842
1763.9761451593388 -84.31106499653684
-----
2100.381924762386 32.60086003301194
2039.658056947729 -36.301744780502304
-----
3074.778927962615 39.384573802906736
2985.171005488369 -50.455454310794316
-----
3022.6197448013713 59.75841186956274
2883.2070963862698 -68.28945455539613
-----
3599.711982